In [2]:
sys.executable

'c:\\Python310\\python.exe'

### packages

In [1]:
import pandas as pd
import sqlite3
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import json
import os
import sys


### basic filtering on both files (not for final product)

In [21]:
json_raw_file = "data/uniprotkb_aeromonas_AND_length_801_TO_2023_11_05.json" #json input file from uniprot download
json_filtered_file = "data/json_filtered_file.json" #json output file for filtered proteins
fasta_raw_file = "data/uniprotkb_aeromonas_AND_length_801_TO_2023_11_05.fasta" #fasta input file from uniprot download
fasta_filtered_file = "data/fasta_filtered_file.fasta" #fasta output file for filtered proteins

In [22]:
import json
import re

# Compiled regular expression for performance
x_re = re.compile(r"X", re.IGNORECASE)

# Loading the data directly without a separate results variable
with open(json_raw_file, 'r') as file:
    results = json.load(file)["results"]

filtered_results = [result for result in results if int(result["annotationScore"]) > 2 and not x_re.search(result["sequence"]["value"])]

with open(json_filtered_file, 'w') as json_file:
    json.dump({"results": filtered_results}, json_file, indent=4)

2487


In [23]:
# Reading the FASTA file
in_fasta_proteins = list(SeqIO.parse(fasta_raw_file, "fasta"))

# Extracting protein IDs from json_results and converting to a set for efficient lookups
protein_ids = set(result["primaryAccession"] for result in filtered_results)

# Filtering in_fasta_proteins based on protein_ids
out_fasta_proteins = [protein for protein in in_fasta_proteins if protein.id.split("|")[1].strip() in protein_ids]

# Writing the filtered proteins to a new FASTA file
with open(fasta_filtered_file, "w") as output_handle:
    SeqIO.write(out_fasta_proteins, output_handle, "fasta")

2487
2487


### populate proteins table

bunu hemen göndermeye gerek yok. gereksiz şekilde kullanmayacağımız fastaları vs. gönderiyoruz. sadece modelleri üretirken. son üründe hepsini göndermek gerekiyor tabii ki.

In [26]:
# Connect to the SQLite database (it will be created if it doesn't exist)
conn = sqlite3.connect('data/aeromonas1.db')
cur = conn.cursor()

# drop proteins table
cur.execute('''DROP TABLE IF EXISTS proteins''')

# Create the table with the necessary columns
cur.execute('''CREATE TABLE IF NOT EXISTS proteins (
    seq_id TEXT PRIMARY KEY,
    description TEXT,
    sequence TEXT,
    length INTEGER,
    molecular_weight REAL,
    instability_index REAL,
    isoelectric_point REAL,
    gravy REAL,
    amino_count TEXT,
    aromaticity REAL,
    flexibility TEXT,
    secondary_structure_fraction TEXT,
    molar_extinction_coefficient TEXT
)''')

conn.close()

In [29]:
# Connect to the SQLite database (it will be created if it doesn't exist)
conn = sqlite3.connect('data/aeromonas1.db')
cur = conn.cursor()

def insert_proteins(records):
    # Prepare bulk insert statements
    cur.executemany('''INSERT INTO proteins (seq_id, description, sequence, length, molecular_weight, 
    instability_index, isoelectric_point, gravy, amino_count, aromaticity, flexibility, 
    secondary_structure_fraction, molar_extinction_coefficient) 
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', records)

# It's often a good idea to disable journaling and synchronous writes for large bulk inserts.
# Don't forget to enable them again if needed for your application's integrity requirements.
conn.execute('PRAGMA journal_mode = OFF')
conn.execute('PRAGMA synchronous = OFF')

# You can also increase cache size
conn.execute('PRAGMA cache_size = 3000') # Adjust the cache size to your needs.

# Collect records in batches
batch_size = 500  # You can tune this number
batch_records = []

with open(fasta_filtered_file, 'r') as fasta_file:
    for record in SeqIO.parse(fasta_file, 'fasta'):
        sequence = str(record.seq)
        seq_id = record.id.split("|")[1].strip()
        length = len(sequence)
        description = record.description

        placeholders = (None,) * 9

        if "X" in sequence or "U" in sequence:
            batch_records.append((seq_id, description, sequence, length) + placeholders)
        else:
            a_seq = ProteinAnalysis(sequence)
            m_weight = a_seq.molecular_weight()
            instab_index = a_seq.instability_index()
            isoele_point = a_seq.isoelectric_point()
            gravy = a_seq.gravy()
            amino_count = a_seq.count_amino_acids()
            amino_count_json = json.dumps(amino_count)
            aromaticity = a_seq.aromaticity()
            flexibility = a_seq.flexibility()
            flexibility_json = json.dumps(flexibility)
            sec_struct_frac = a_seq.secondary_structure_fraction()
            sec_struct_frac_json = json.dumps(sec_struct_frac)
            ext_coeff = a_seq.molar_extinction_coefficient()
            ext_coeff_json = json.dumps(ext_coeff)

            batch_records.append((seq_id, description, sequence, length, m_weight, instab_index, isoele_point, gravy, amino_count_json, aromaticity, flexibility_json, sec_struct_frac_json, ext_coeff_json))
        
        # Insert in batches
        if len(batch_records) >= batch_size:
            insert_proteins(batch_records)
            batch_records = []  # Reset batch

# Insert any remaining records
if batch_records:
    insert_proteins(batch_records)

# Commit and clean up
conn.commit()
conn.execute('PRAGMA journal_mode = DELETE') # Set it back to default or your preferred mode
conn.execute('PRAGMA synchronous = NORMAL') # Set it back to default or your preferred mode
conn.close()

### populate metadata table

In [33]:
conn = sqlite3.connect('data/aeromonas1.db')
cur = conn.cursor()

# Drop the 'metadata' table if it exists
cur.execute('''DROP TABLE IF EXISTS metadata''')
conn.commit()  # Commit the changes

# Create the 'metadata' table
cur.execute('''
    CREATE TABLE metadata (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        primaryAccession TEXT,
        annotationScore INTEGER,
        comments TEXT,
        keywords TEXT,
        uniProtKBCrossReferences TEXT
    )
''')
conn.commit()  # Commit the changes

#close connection
conn.close()


In [34]:
import sqlite3
import json

def insert_records(records):
    # Prepare the data for insertion, ensuring that all list or dict types are converted to JSON strings
    prepared_records = []
    for record in records:
        prepared_record = tuple(
            json.dumps(item) if isinstance(item, (list, dict)) else item
            for item in record
        )
        prepared_records.append(prepared_record)

    # Use executemany to insert all records in one go
    cur.executemany('''
        INSERT INTO metadata (
            primaryAccession, 
            annotationScore, 
            comments, 
            keywords, 
            uniProtKBCrossReferences
        ) VALUES (?, ?, ?, ?, ?)
    ''', prepared_records)
    conn.commit()

# Begin the transaction
conn = sqlite3.connect('data/aeromonas1.db')
cur = conn.cursor()

# Disable journaling and synchronous writes for the duration of the insert operation
cur.execute('PRAGMA journal_mode = OFF')
cur.execute('PRAGMA synchronous = OFF')
cur.execute('PRAGMA cache_size = 1000')  # Increase cache size

# Assuming 'your_json_file.json' is in the same directory as your script
with open(json_filtered_file, 'r') as f:
    data = json.load(f)

batch_size = 1000  # Define a suitable batch size
batch_records = []

results = data.get('results', [])

for item in results:
    record = (
        item.get('primaryAccession'),
        item.get('annotationScore'),
        json.dumps(item.get('comments')) if isinstance(item.get('comments'), list) else item.get('comments'),
        json.dumps(item.get('keywords')) if isinstance(item.get('keywords'), list) else item.get('keywords'),
        json.dumps(item.get('uniProtKBCrossReferences')) if isinstance(item.get('uniProtKBCrossReferences'), list) else item.get('uniProtKBCrossReferences'),
    )
    batch_records.append(record)
    
    if len(batch_records) >= batch_size:
        insert_records(batch_records)
        batch_records = []  # Reset the batch

# Insert any remaining records
if batch_records:
    insert_records(batch_records)

# Commit the transaction
conn.commit()

# Re-enable journaling and synchronous writes
cur.execute('PRAGMA journal_mode = DELETE')  # Set it back to the default or your preferred mode
cur.execute('PRAGMA synchronous = NORMAL')   # Set it back to the default or your preferred mode

# Close the connection
conn.close()


### send iFeature tables

In [35]:
# Create a SQLite database connection
conn = sqlite3.connect('data/aeromonas1.db')

# Specify the path to the directory containing your TSV files
tsv_directory = 'data/iFeature/'

# Iterate through each TSV file in the directory
for tsv_filename in os.listdir(tsv_directory):
    if tsv_filename.endswith('.tsv'):
        # Construct the full file path
        file_path = os.path.join(tsv_directory, tsv_filename)
        
        # Read the TSV file into a DataFrame
        df = pd.read_csv(file_path, sep='\t', encoding='utf-8')

        # Extract protein_id and remove the '#' column
        df['protein_id'] = df['#'].apply(lambda x: x.split("|")[1].strip())
        
        # Get the file name without the .tsv extension
        file_name = tsv_filename[:-4]
        
        # Add the file name as a prefix to all columns except 'protein_id'
        df.columns = [f"{file_name}_{col}" if col != 'protein_id' else col for col in df.columns]
        
        # Use 'to_sql' method to write the DataFrame to the SQLite table
        df.to_sql(file_name, conn, if_exists='replace', index=False)  # Use 'append' if you want to add to an existing table

# Close the connection
conn.close()

### merge all

In [39]:
# Connect to the SQLite database (it will be created if it doesn't exist)
conn = sqlite3.connect('data/aeromonas1.db')
cur = conn.cursor()

# Create the table with the necessary columns
cur.execute('''
CREATE TABLE new_merged_data_filtered AS
SELECT
    m.*,
    p.*,
    CTDC.*,
    CTDD.*,
    CTDT.*,
    CTriad.*,
    DPC.*,
    GAAC.*
FROM
    metadata m
    LEFT JOIN proteins p ON m.primaryAccession = p.seq_id
    LEFT JOIN CTDC ON m.primaryAccession = CTDC.protein_id
    LEFT JOIN CTDD ON m.primaryAccession = CTDD.protein_id
    LEFT JOIN CTDT ON m.primaryAccession = CTDT.protein_id
    LEFT JOIN CTriad ON m.primaryAccession = CTriad.protein_id
    LEFT JOIN DPC ON m.primaryAccession = DPC.protein_id
    LEFT JOIN GAAC ON m.primaryAccession = GAAC.protein_id;
''')

#drop tables
cur.execute('''DROP TABLE IF EXISTS CTDC''')
cur.execute('''DROP TABLE IF EXISTS CTDD''')
cur.execute('''DROP TABLE IF EXISTS CTDT''')
cur.execute('''DROP TABLE IF EXISTS CTriad''')
cur.execute('''DROP TABLE IF EXISTS DPC''')
cur.execute('''DROP TABLE IF EXISTS GAAC''')
cur.execute('''DROP TABLE IF EXISTS proteins''')
cur.execute('''DROP TABLE IF EXISTS metadata''')


conn.close()

### some filtering (son ürün için geçerli)

In [3]:
# run sql query using pandas on a database

# Create a SQLite database connection
conn = sqlite3.connect('data/aeromonas1.db')

# Define the SQL query
query = '''
---sql
SELECT COUNT(*)
FROM new_merged_data_filtered
WHERE annotationScore IN (3, 4, 5)
AND sequence NOT LIKE '%X%'
AND uniProtKBCrossReferences LIKE '%GO:%'
AND (
  uniProtKBCrossReferences LIKE '%"id": "GO:0051287"%' OR
  uniProtKBCrossReferences LIKE '%"id": "GO:0000287"%' OR
  uniProtKBCrossReferences LIKE '%"id": "GO:0003677"%' OR
  uniProtKBCrossReferences LIKE '%"id": "GO:0003723"%' OR
  uniProtKBCrossReferences LIKE '%"id": "GO:0005506"%' OR
  uniProtKBCrossReferences LIKE '%"id": "GO:0005524"%' OR
  uniProtKBCrossReferences LIKE '%"id": "GO:0005525"%' OR
  uniProtKBCrossReferences LIKE '%"id": "GO:0008270"%' OR
  uniProtKBCrossReferences LIKE '%"id": "GO:0016887"%' OR
  uniProtKBCrossReferences LIKE '%"id": "GO:0019843"%' OR
  uniProtKBCrossReferences LIKE '%"id": "GO:0030170"%' OR
  uniProtKBCrossReferences LIKE '%"id": "GO:0046872"%' OR
  uniProtKBCrossReferences LIKE '%"id": "GO:0050661"%' OR
  uniProtKBCrossReferences LIKE '%"id": "GO:0051287"%' OR
  uniProtKBCrossReferences LIKE '%"id": "GO:0051539"%'
)
'''

# Execute the query
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

df

,COUNT(*)
0,2224


### final table

In [43]:
conn = sqlite3.connect('data/aeromonas1.db')
cur = conn.cursor()

cur.executescript('''
---sql
CREATE TABLE new_merged_data_filtered AS
SELECT
    *,
    (
      SELECT json_group_array(json_extract(loc.value, '$.location.value'))
      FROM json_each(json_extract(t.comments, '$')) AS entry
      LEFT JOIN json_each(json_extract(entry.value, '$.subcellularLocations')) AS loc
      WHERE json_extract(entry.value, '$.commentType') = 'SUBCELLULAR LOCATION'
    ) AS location_values
FROM new_merged_data_filtered AS t;

---sql
drop table new_merged_data_filtered;

ALTER TABLE new_merged_data_filtered ADD COLUMN cytoplasm INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN cell_membrane INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN cell_wall INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN secreted INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN periplasm INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN cell_surface INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN cell_envelope INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN chlorosome INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN cellular_thylakoid_membrane INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN cellular_chromatopore_membrane INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN single_pass_membrane_protein INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN multi_pass_membrane_protein INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN peripheral_membrane_protein INTEGER;

---sql
UPDATE new_merged_data_filtered
SET
  cytoplasm = CASE 
    WHEN location_values LIKE '%"Cytoplasm"%' 
      OR location_values LIKE '%"Cytoplasmic side"%' 
      OR location_values LIKE '%"Cytoplasm, nucleoid"%' THEN 1 ELSE 0 END,
  cell_membrane = CASE 
    WHEN location_values LIKE '%"Cell membrane"%' 
      OR location_values LIKE '%"Membrane"%' 
      OR location_values LIKE '%"Cell inner membrane"%' 
      OR location_values LIKE '%"Cell outer membrane"%' THEN 1 ELSE 0 END,
  cell_wall = CASE WHEN location_values LIKE '%"Cell wall"%' THEN 1 ELSE 0 END,
  secreted = CASE WHEN location_values LIKE '%"Secreted"%' THEN 1 ELSE 0 END,
  periplasm = CASE 
    WHEN location_values LIKE '%"Periplasm"%' 
      OR location_values LIKE '%"Periplasmic side"%' THEN 1 ELSE 0 END,
  cell_surface = CASE WHEN location_values LIKE '%"Cell surface"%' THEN 1 ELSE 0 END,
  cell_envelope = CASE WHEN location_values LIKE '%"Cell envelope"%' THEN 1 ELSE 0 END,
  chlorosome = CASE WHEN location_values LIKE '%"Chlorosome"%' THEN 1 ELSE 0 END,
  cellular_thylakoid_membrane = CASE WHEN location_values LIKE '%"Cellular thylakoid membrane"%' THEN 1 ELSE 0 END,
  cellular_chromatopore_membrane = CASE WHEN location_values LIKE '%"Cellular chromatopore membrane"%' THEN 1 ELSE 0 END,
  single_pass_membrane_protein = CASE WHEN location_values LIKE '%"Single-pass membrane protein"%' THEN 1 ELSE 0 END,
  multi_pass_membrane_protein = CASE WHEN location_values LIKE '%"Multi-pass membrane protein"%' THEN 1 ELSE 0 END,
  peripheral_membrane_protein = CASE WHEN location_values LIKE '%"Peripheral membrane protein"%' THEN 1 ELSE 0 END;

''')

conn.commit()

conn.close()

In [44]:
conn = sqlite3.connect('data/aeromonas1.db')
cur = conn.cursor()

cur.executescript('''
--- data binding
ALTER TABLE new_merged_data_filtered ADD COLUMN dna_binding INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN rna_binding INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN metal_binding INTEGER;

UPDATE c
SET
  dna_binding = CASE WHEN keywords LIKE '%"DNA-binding"%' THEN 1 ELSE 0 END,
  rna_binding = CASE WHEN keywords LIKE '%"RNA-binding"%' THEN 1 ELSE 0 END,
  metal_binding = CASE WHEN keywords LIKE '%"Metal-binding"%' THEN 1 ELSE 0 END;

''')

conn.commit()

conn.close()

In [47]:
conn = sqlite3.connect('data/aeromonas1.db')
cur = conn.cursor()

cur.executescript('''
ALTER TABLE new_merged_data_filtered ADD COLUMN GO_0051287 INTEGER DEFAULT 0;
ALTER TABLE new_merged_data_filtered ADD COLUMN GO_0009331 INTEGER DEFAULT 0;
ALTER TABLE new_merged_data_filtered ADD COLUMN GO_0003677 INTEGER DEFAULT 0;
ALTER TABLE new_merged_data_filtered ADD COLUMN GO_0003723 INTEGER DEFAULT 0;
ALTER TABLE new_merged_data_filtered ADD COLUMN GO_0005506 INTEGER DEFAULT 0;
ALTER TABLE new_merged_data_filtered ADD COLUMN GO_0005524 INTEGER DEFAULT 0;
ALTER TABLE new_merged_data_filtered ADD COLUMN GO_0046167 INTEGER DEFAULT 0;
ALTER TABLE new_merged_data_filtered ADD COLUMN GO_0008270 INTEGER DEFAULT 0;
ALTER TABLE new_merged_data_filtered ADD COLUMN GO_0016887 INTEGER DEFAULT 0;
ALTER TABLE new_merged_data_filtered ADD COLUMN GO_0019843 INTEGER DEFAULT 0;
ALTER TABLE new_merged_data_filtered ADD COLUMN GO_0008654 INTEGER DEFAULT 0;
ALTER TABLE new_merged_data_filtered ADD COLUMN GO_0046872 INTEGER DEFAULT 0;
ALTER TABLE new_merged_data_filtered ADD COLUMN GO_0050661 INTEGER DEFAULT 0;
ALTER TABLE new_merged_data_filtered ADD COLUMN GO_0051539 INTEGER DEFAULT 0;


UPDATE new_merged_data_filtered
SET
  GO_0051287 = CASE WHEN uniProtKBCrossReferences LIKE '%"GO:0051287"%' THEN 1 ELSE 0 END,
  GO_0009331 = CASE WHEN uniProtKBCrossReferences LIKE '%"GO:0009331"%' THEN 1 ELSE 0 END,
  GO_0003677 = CASE WHEN uniProtKBCrossReferences LIKE '%"GO:0003677"%' THEN 1 ELSE 0 END,
  GO_0003723 = CASE WHEN uniProtKBCrossReferences LIKE '%"GO:0003723"%' THEN 1 ELSE 0 END,
  GO_0005506 = CASE WHEN uniProtKBCrossReferences LIKE '%"GO:0005506"%' THEN 1 ELSE 0 END,
  GO_0005524 = CASE WHEN uniProtKBCrossReferences LIKE '%"GO:0005524"%' THEN 1 ELSE 0 END,
  GO_0046167 = CASE WHEN uniProtKBCrossReferences LIKE '%"GO:0046167"%' THEN 1 ELSE 0 END,
  GO_0008270 = CASE WHEN uniProtKBCrossReferences LIKE '%"GO:0008270"%' THEN 1 ELSE 0 END,
  GO_0016887 = CASE WHEN uniProtKBCrossReferences LIKE '%"GO:0016887"%' THEN 1 ELSE 0 END,
  GO_0019843 = CASE WHEN uniProtKBCrossReferences LIKE '%"GO:0019843"%' THEN 1 ELSE 0 END,
  GO_0008654 = CASE WHEN uniProtKBCrossReferences LIKE '%"GO:0008654"%' THEN 1 ELSE 0 END,
  GO_0046872 = CASE WHEN uniProtKBCrossReferences LIKE '%"GO:0046872"%' THEN 1 ELSE 0 END,
  GO_0050661 = CASE WHEN uniProtKBCrossReferences LIKE '%"GO:0050661"%' THEN 1 ELSE 0 END,
  GO_0051539 = CASE WHEN uniProtKBCrossReferences LIKE '%"GO:0051539"%' THEN 1 ELSE 0 END;
''')

conn.commit()

conn.close()

In [48]:
conn = sqlite3.connect('data/aeromonas1.db')
cur = conn.cursor()

cur.executescript('''

ALTER TABLE new_merged_data_filtered ADD COLUMN A INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN C INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN D INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN E INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN F INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN G INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN H INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN I INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN K INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN L INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN M INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN N INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN P INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN Q INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN R INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN S INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN T INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN V INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN W INTEGER;
ALTER TABLE new_merged_data_filtered ADD COLUMN Y INTEGER;

UPDATE new_merged_data_filtered
SET 
    A = json_extract(amino_count, '$.A'),
    C = json_extract(amino_count, '$.C'),
    D = json_extract(amino_count, '$.D'),
    E = json_extract(amino_count, '$.E'),
    F = json_extract(amino_count, '$.F'),
    G = json_extract(amino_count, '$.G'),
    H = json_extract(amino_count, '$.H'),
    I = json_extract(amino_count, '$.I'),
    K = json_extract(amino_count, '$.K'),
    L = json_extract(amino_count, '$.L'),
    M = json_extract(amino_count, '$.M'),
    N = json_extract(amino_count, '$.N'),
    P = json_extract(amino_count, '$.P'),
    Q = json_extract(amino_count, '$.Q'),
    R = json_extract(amino_count, '$.R'),
    S = json_extract(amino_count, '$.S'),
    T = json_extract(amino_count, '$.T'),
    V = json_extract(amino_count, '$.V'),
    W = json_extract(amino_count, '$.W'),
    Y = json_extract(amino_count, '$.Y');

    ''')

conn.commit()

conn.close()

In [56]:
conn = sqlite3.connect('data/aeromonas1.db')
cur = conn.cursor()

cur.executescript('''

ALTER TABLE new_merged_data_filtered DROP COLUMN primaryAccession;
ALTER TABLE new_merged_data_filtered DROP COLUMN annotationScore;
ALTER TABLE new_merged_data_filtered DROP COLUMN keywords;
ALTER TABLE new_merged_data_filtered DROP COLUMN uniProtKBCrossReferences;
ALTER TABLE new_merged_data_filtered DROP COLUMN seq_id;
ALTER TABLE new_merged_data_filtered DROP COLUMN description;
ALTER TABLE new_merged_data_filtered DROP COLUMN sequence;
ALTER TABLE new_merged_data_filtered DROP COLUMN amino_count;
ALTER TABLE new_merged_data_filtered DROP COLUMN "CTDC_#";
ALTER TABLE new_merged_data_filtered DROP COLUMN "protein_id";
ALTER TABLE new_merged_data_filtered DROP COLUMN "CTDD_#";
ALTER TABLE new_merged_data_filtered DROP COLUMN "protein_id:1";
ALTER TABLE new_merged_data_filtered DROP COLUMN "CTDT_#";
ALTER TABLE new_merged_data_filtered DROP COLUMN "protein_id:2";
ALTER TABLE new_merged_data_filtered DROP COLUMN "CTriad_#";
ALTER TABLE new_merged_data_filtered DROP COLUMN "protein_id:3";
ALTER TABLE new_merged_data_filtered DROP COLUMN "DPC_#";
ALTER TABLE new_merged_data_filtered DROP COLUMN "protein_id:4";
ALTER TABLE new_merged_data_filtered DROP COLUMN "GAAC_#";
ALTER TABLE new_merged_data_filtered DROP COLUMN "protein_id:2682086651";
ALTER TABLE new_merged_data_filtered DROP COLUMN location_values;
ALTER TABLE new_merged_data_filtered DROP COLUMN flexibility;
ALTER TABLE new_merged_data_filtered DROP COLUMN secondary_structure_fraction;
ALTER TABLE new_merged_data_filtered DROP COLUMN molar_extinction_coefficient;
''')

conn.commit()

conn.close()

In [57]:
conn = sqlite3.connect('data/aeromonas1.db')
cur = conn.cursor()

cur.executescript('''

ALTER TABLE new_merged_data_filtered DROP COLUMN flexibility;
ALTER TABLE new_merged_data_filtered DROP COLUMN secondary_structure_fraction;
ALTER TABLE new_merged_data_filtered DROP COLUMN molar_extinction_coefficient;

''')

conn.commit()

conn.close()

### create model and save as pickle

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
from skmultilearn.adapt import MLkNN

import pickle


from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from scipy.sparse import csr_matrix

In [4]:
# read data from database to pandas dataframe

conn = sqlite3.connect('data/aeromonas1.db')
cur = conn.cursor()

df = pd.read_sql_query("SELECT * FROM new_merged_data_filtered", conn)

In [5]:
X = df[df.columns[~df.columns.str.startswith('GO_')]]
y = df[df.columns[df.columns.str.startswith('GO_')]]

In [ ]:
def train_and_save_model(model, X, y, cv, model_name):
    # Suppress FutureWarnings
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)

        # Perform cross-validation
        accuracy = np.mean(cross_val_score(model, X, y, cv=cv))
        print(f"{model_name} Cross-Validation Accuracy: {accuracy}")

    # Train the model on the entire dataset
    model.fit(X, y)

    # Save the model to a pickle file
    with open(f'models/{model_name}.pkl', 'wb') as file:
        pickle.dump(model, file)

    print(f"Model saved as {model_name}.pkl")

# Assuming X and y are your features and labels
# Define your models
models = {
    "RandomForestClassifier": RandomForestClassifier(n_estimators=100, random_state=0),
    #"LogisticRegression": LogisticRegression(random_state=0),
    #"SVC": SVC(random_state=0),
    #"GradientBoostingClassifier": GradientBoostingClassifier(n_estimators=100, random_state=0),
    "gb_model": MultiOutputClassifier(HistGradientBoostingClassifier(random_state=0)),
    # "mlknn_model": MLkNN(k=10)
}

if isinstance(y, pd.DataFrame):
    y_sparse = csr_matrix(y.values)
else:
    y_sparse = y  # Assuming y is already in sparse format

# Train and save each model
for model_name, model in models.items():
    train_and_save_model(model, X, y, cv=10, model_name=model_name)


### predictions

In [ ]:
import pickle

# Load the model from file
with open('models/gb_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

# Assuming you have input data prepared as X_test
# X_test = ... # Your code to prepare the input data

# Make predictions
predictions = loaded_model.predict(X)

# Output or use the predictions
predictions

In [ ]:
predictions_df = pd.DataFrame(predictions, columns=y.columns)
predictions_df

# give sum of each column
predictions_df.sum(axis=0)